In [10]:
import yfinance as yf
import finplot as fplt
import numpy as np
import pandas as pd
import mysql.connector
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="1234",
  database="bolsa_valores"
)
cursor = conn.cursor()

chamada_api = pd.read_sql('SELECT * FROM cotacao', conn)
chamada_api = chamada_api.loc[chamada_api['SYMBOL'] == 'PETR4']


def plot_accumulation_distribution(df, ax):
    ad = (2*df.CLOSE-df.HIGH-df.LOW) * df.VOLUME / (df.HIGH - df.LOW)
    ad.cumsum().ffill().plot(ax=ax, legend='Accum/Dist', color='#f00000')


def plot_bollinger_bands(df, ax):
    mean = df.CLOSE.rolling(20).mean()
    stddev = df.CLOSE.rolling(20).std()
    df['boll_hi'] = mean + 2.5*stddev
    df['boll_lo'] = mean - 2.5*stddev
    p0 = df.boll_hi.plot(ax=ax, color='#808080', legend='BB')
    p1 = df.boll_lo.plot(ax=ax, color='#808080')
    fplt.fill_between(p0, p1, color='#1A1C1D')


def plot_ema(df, ax):
    df.CLOSE.ewm(span=9).mean().plot(ax=ax, legend='EMA',color = '#eef')


def plot_rsi(df, ax):
    diff = df.CLOSE.diff().values
    gains = diff
    losses = -diff
    with np.errstate(invalid='ignore'):
        gains[(gains<0)|np.isnan(gains)] = 0.0
        losses[(losses<=0)|np.isnan(losses)] = 1e-10 
    n = 14
    m = (n-1) / n
    ni = 1 / n
    g = gains[n] = np.nanmean(gains[:n])
    l = losses[n] = np.nanmean(losses[:n])
    gains[:n] = losses[:n] = np.nan
    for i,v in enumerate(gains[n:],n):
        g = gains[i] = ni*v + m*g
    for i,v in enumerate(losses[n:],n):
        l = losses[i] = ni*v + m*l
    rs = gains / losses
    df['rsi'] = 100 - (100/(1+rs))
    df.rsi.plot(ax=ax, legend='RSI')
    fplt.set_y_range(-20, 120, ax=ax)
    fplt.add_band(3, 97, ax=ax, color='#1A1C1D')


def plot_moving_avg(df, ax):
    ma20 = df.CLOSE.rolling(20).mean()
    ma50 = df.CLOSE.rolling(50).mean()
    fplt.plot(ma20, legend = "MA-20", ax=ax)
    fplt.plot(ma50, legend = "MA-50", ax=ax)


def plot_candles(df, ax):
    candles = df[['OPEN', 'CLOSE', 'HIGH', 'LOW']]
    daily_plot = fplt.candlestick_ochl(candles, candle_width=1)
    daily_plot.colors.update(dict(bull_body='#bfb', bull_shadow='#ada', bear_body='#fbc', bear_shadow='#dab'))


def plot_volume(df, ax):
    df_renko = chamada_api.reset_index()
    df_renko["DATE"] = pd.to_datetime(df_renko['DATE'])
    fplt.volume_ocv(df_renko[['DATE','OPEN','CLOSE','VOLUME']], ax=ax)
    fplt.plot(df_renko.VOLUME.ewm(span=24).mean(), ax=ax, color='#eef', legend='Volume')

def grafico(df):
    ax,ax2,ax3,ax4 = fplt.create_plot(title='Gráfico', rows = 4, maximize=True)
    b = fplt.background = fplt.odd_plot_background = '#010101'
    w = fplt.foreground = '#eef'
    fplt.cross_hair_color = w+'a'

    plot_candles(chamada_api, ax)
    plot_volume(chamada_api, ax2)
    plot_bollinger_bands(chamada_api, ax)
    plot_ema(chamada_api, ax)
    plot_accumulation_distribution(chamada_api, ax3)
    plot_rsi(chamada_api, ax4)
    plot_moving_avg(chamada_api, ax)
    fplt.show()

grafico(chamada_api)


C:\Users\igorb\AppData\Local\Temp\ipykernel_23240\2413477934.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chamada_api = pd.read_sql('SELECT * FROM cotacao', conn)


In [12]:
#media de volume
chamada_api['m_vol'] = chamada_api.VOLUME.ewm(span=24).mean()

#medias móveis
chamada_api['ma20'] = chamada_api.CLOSE.rolling(20).mean()
chamada_api['ma50'] = chamada_api.CLOSE.rolling(50).mean()

#bollinger bands
mean = chamada_api.CLOSE.rolling(20).mean()
stddev = chamada_api.CLOSE.rolling(20).std()
chamada_api['boll_hi'] = mean + 2.5*stddev
chamada_api['boll_lo'] = mean - 2.5*stddev

#acumulação e distribuição
ad = (2*chamada_api.CLOSE-chamada_api.HIGH-chamada_api.LOW) * chamada_api.VOLUME / (chamada_api.HIGH - chamada_api.LOW)

#indice relativo
diff = chamada_api.CLOSE.diff().values
gains = diff
losses = -diff
with np.errstate(invalid='ignore'):
    gains[(gains<0)|np.isnan(gains)] = 0.0
    losses[(losses<=0)|np.isnan(losses)] = 1e-10 
n = 14
m = (n-1) / n
ni = 1 / n
g = gains[n] = np.nanmean(gains[:n])
l = losses[n] = np.nanmean(losses[:n])
gains[:n] = losses[:n] = np.nan
for i,v in enumerate(gains[n:],n):
    g = gains[i] = ni*v + m*g
for i,v in enumerate(losses[n:],n):
    l = losses[i] = ni*v + m*l
rs = gains / losses
chamada_api['rsi'] = 100 - (100/(1+rs))


chamada_api


,DATE,SYMBOL,OPEN,HIGH,LOW,CLOSE,VOLUME,DIVIDENDS,boll_hi,boll_lo,rsi,m_vol,ma20,ma50
131,2020-03-23,PETR4,5.751809,5.785056,5.357589,5.462081,107092400,0.0,NaN,NaN,NaN,1.070924e+08,NaN,NaN
352,2020-03-24,PETR4,5.970291,6.431006,5.832552,6.293266,114259200,0.0,NaN,NaN,NaN,1.108251e+08,NaN,NaN
353,2020-03-24,PETR4,5.970291,6.431006,5.832552,6.293267,114259200,0.0,NaN,NaN,NaN,1.120665e+08,NaN,NaN
582,2020-03-25,PETR4,6.288516,7.024710,6.093781,6.801477,158286200,0.0,NaN,NaN,NaN,1.251041e+08,NaN,NaN
812,2020-03-26,PETR4,6.839474,7.214695,6.559246,6.834724,130618400,0.0,NaN,NaN,NaN,1.263981e+08,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169424,2023-03-23,PETR4,23.370001,23.670000,22.590000,22.799999,74319100,0.0,27.320323,21.701677,34.708527,6.714872e+07,24.5110,25.3114
169634,2023-03-24,PETR4,22.709999,23.129999,22.480000,22.790001,52556800,0.0,27.196777,21.514222,34.646183,6.598137e+07,24.3555,25.2816
169839,2023-03-27,PETR4,23.110001,23.360001,22.830000,23.180000,39148700,0.0,26.913110,21.500890,39.231333,6.383476e+07,24.2070,25.2526
169840,2023-03-27,PETR4,23.110001,23.360001,22.830000,23.260000,34519800,0.0,26.791754,21.424246,40.158788,6.148956e+07,24.1080,25.2264


In [11]:
chamada_api

,DATE,SYMBOL,OPEN,HIGH,LOW,CLOSE,VOLUME,DIVIDENDS,boll_hi,boll_lo,rsi
131,2020-03-23,PETR4,5.751809,5.785056,5.357589,5.462081,107092400,0.0,NaN,NaN,NaN
352,2020-03-24,PETR4,5.970291,6.431006,5.832552,6.293266,114259200,0.0,NaN,NaN,NaN
353,2020-03-24,PETR4,5.970291,6.431006,5.832552,6.293267,114259200,0.0,NaN,NaN,NaN
582,2020-03-25,PETR4,6.288516,7.024710,6.093781,6.801477,158286200,0.0,NaN,NaN,NaN
812,2020-03-26,PETR4,6.839474,7.214695,6.559246,6.834724,130618400,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
169424,2023-03-23,PETR4,23.370001,23.670000,22.590000,22.799999,74319100,0.0,27.320323,21.701677,34.708527
169634,2023-03-24,PETR4,22.709999,23.129999,22.480000,22.790001,52556800,0.0,27.196777,21.514222,34.646183
169839,2023-03-27,PETR4,23.110001,23.360001,22.830000,23.180000,39148700,0.0,26.913110,21.500890,39.231333
169840,2023-03-27,PETR4,23.110001,23.360001,22.830000,23.260000,34519800,0.0,26.791754,21.424246,40.158788
